In [1]:
import pandas as pd # data analysis library
import re # lib for extraxting text

In [2]:
#read all pdf files in the folder ,open them and extract the invoice amount,invoice date,card number...etc.
#avoid manual task of reading opening each PDF manually to get invoice amount and number
import glob
import os
path = r'D:\Docs\total_sep_24' # path to our folder containing all fuel invoices
os.chdir(path)
all_files = glob.glob("*.pdf")
#print(all_files)


In [3]:
from PyPDF2 import PdfReader #library containing methods for extracting text from PDF files
##put file name and amount in a list
list_invoices=[] ## our empty list to store Invoice number and amount.
#Define path to PDF file----we have all our files ...so we loop through them.
for file in all_files:
    #Open the file in binary mode for reading
    with open(file, 'rb') as pdf_file:
        #Read the PDF file
        ##print(pdf_file)
        pdf_reader =PdfReader(pdf_file)    
        page = pdf_reader.pages[0] # we read page one since it has data required
        #Extract text from page one.
        text = page.extract_text()
        #print(text)
        ## here is where regex does the magic...eeeish!
        ## here i use regular expression to extract amount. We look for text starting with USH and followed by numbers.
        ## Printed text,put it in online regex reader to get the required regex
        text_amount = re.findall(r'USH\s\d*\s\d*\s\d*', text) ## this line returns a list of all matches
        ##now get card name. It is where there is Page 1 of something
        text_card=re.findall(r'Page\s\d\/d*\s*\w*\s\w*.*', text)
        date_inv=re.findall(r'\s\d{2}\/\d{2}\/\d{4}', text)
        ##print(date_inv)
        list_invoices.append([file,text_amount[0],text_card[0],date_inv[1]])


In [4]:
## this is how our data looks
list_invoices[:1]

[['FA24162748.PDF', 'USH 39 463 000', 'Page 1/2\nUNRA LUWERO', ' 30/09/2024']]

In [5]:
##we convert into a data frame
list_i=pd.DataFrame(list_invoices,columns=['Invoice_nu','Amount','card','inv_date'])
###remove .pdf from the invoice number
list_i['Invoice_nu']=list_i['Invoice_nu'].str[:10]
list_i.head()

,Invoice_nu,Amount,card,inv_date
0,FA24162748,USH 39 463 000,Page 1/2\nUNRA LUWERO,30/09/2024
1,FA24162763,USH 54 230 000,Page 1/2\nUNRA KASESE,30/09/2024
2,FA24162784,USH 731 175 0731,Page 1/2\nPANYAMUR- WANSEKO FERRY,30/09/2024
3,FA24162785,USH 59 844 000,Page 1/2\nUNRA LIRA,30/09/2024
4,FA24162799,USH 68 831 000,Page 1/2\nUNRA SIGULU FERRY,30/09/2024


In [6]:
## remove USH from the amount
list_i['Amount']=list_i['Amount'].str.replace("USH",'').replace(" ",'')

In [7]:
list_i.head()

,Invoice_nu,Amount,card,inv_date
0,FA24162748,39 463 000,Page 1/2\nUNRA LUWERO,30/09/2024
1,FA24162763,54 230 000,Page 1/2\nUNRA KASESE,30/09/2024
2,FA24162784,731 175 0731,Page 1/2\nPANYAMUR- WANSEKO FERRY,30/09/2024
3,FA24162785,59 844 000,Page 1/2\nUNRA LIRA,30/09/2024
4,FA24162799,68 831 000,Page 1/2\nUNRA SIGULU FERRY,30/09/2024


In [8]:
##let us remove white space by use of regex again...it is usually stubborn
def remove(string):
    pattern = re.compile(r'\s+')
    return re.sub(pattern, '', string)

In [9]:
list_i['inv_date']=list_i['inv_date'].apply(remove)

In [10]:
##let us remove page of....
def remove_page(string):
    pattern = re.compile(r'Page\s\d\/\d')
    return re.sub(pattern, '', string)

In [11]:
#3 we now aply our regex function to our column
list_i['Amount']=list_i['Amount'].apply(remove)


In [12]:
#3 we now remove page
list_i['card']=list_i['card'].apply(remove_page)

In [13]:
def remove_n(string):
    pattern = re.compile(r'/\n')
    return re.sub(pattern, '', string)

In [14]:
list_i['card']=list_i['card'].apply(remove_n)

In [15]:
list_i.head()

,Invoice_nu,Amount,card,inv_date
0,FA24162748,39463000,\nUNRA LUWERO,30/09/2024
1,FA24162763,54230000,\nUNRA KASESE,30/09/2024
2,FA24162784,7311750731,\nPANYAMUR- WANSEKO FERRY,30/09/2024
3,FA24162785,59844000,\nUNRA LIRA,30/09/2024
4,FA24162799,68831000,\nUNRA SIGULU FERRY,30/09/2024


In [16]:
## we convert into numbers to enable us do addition
list_i['Amount']=list_i['Amount'].astype(float)

In [17]:
list_i

,Invoice_nu,Amount,card,inv_date
0,FA24162748,3.946300e+07,\nUNRA LUWERO,30/09/2024
1,FA24162763,5.423000e+07,\nUNRA KASESE,30/09/2024
2,FA24162784,7.311751e+09,\nPANYAMUR- WANSEKO FERRY,30/09/2024
3,FA24162785,5.984400e+07,\nUNRA LIRA,30/09/2024
4,FA24162799,6.883100e+07,\nUNRA SIGULU FERRY,30/09/2024
5,FA24162810,2.020000e+09,\nUNRA- MV BUGALA,30/09/2024
6,FA24162861,3.158350e+09,\nIAN TUMWIJUKYE,30/09/2024
7,FA24162885,4.859000e+07,\nUNRA SOROTI,30/09/2024
8,FA24162886,3.535000e+07,\nUNRA MASAKA,30/09/2024
9,FA24162887,2.797329e+07,\nUNRA JINJA,30/09/2024


In [18]:
list_i['card']=list_i['card'].str.replace('\n','')

In [19]:
list_i.tail()

,Invoice_nu,Amount,card,inv_date
28,FA24163159,8.940800e+06,LAROPI FERRY,30/09/2024
29,FA24163160,2.100000e+07,UNRA MOROTO,30/09/2024
30,FA24163161,8.309550e+07,UNRA KABALE,30/09/2024
31,FA24163184,3.542000e+09,RACHEAL NGONZEBWA,30/09/2024
32,FA24163185,3.401300e+07,UNRA MPIGI,30/09/2024


In [20]:
## we then save it as an excel file!
list_i.to_excel(r'D:\Docs\total_invoices_sep_24.xlsx',index=None)